In [3]:
from portfolio_optimization.optimization.GeneralOptimization import (
    GeneralOptimizationDelegate,
)
from portfolio_optimization.optimization.vault_allocation import VaultAllocation
import numpy as np
import pandas as pd
import seaborn as sns

# Yields
mu = pd.Series(
    [4 / 100, 4 / 100, 4 / 100]
)

print("mu (%):", np.round(mu * 100, 2))
print(" ")

# vector of recovery rates
R = pd.Series(
    [50 / 100, 50 / 100, 50 / 100]
)


print("Recovery (%):", np.round(R * 100, 2))
print(" ")

# vector of probability of default
lmb = pd.Series(
    [50 / 100, 50 / 100, 50 / 100]
)

print("lambda (%):", np.round(lmb * 100, 2))
print(" ")

# The parameter omega represents the risk aversion of the strategy and ranges from 0% to 100%
# An allocation achieved at 100% would suit a risk-averse investor,
# whereas one achieved at 0% would primarily focus on maximizing yield.
omega = 60 / 100

EPSILON_RANGE = 1/100/100 # (to avoid numerical issues at the boundaries)
# The diversification factor D ranges from 1/n% * (1 + EPSILON_RANGE) to 100% * (1 - EPSILON_RANGE)
minimumD = 1 / (len(mu) / 100) * (1 + EPSILON_RANGE)
maximumD = 100 * (1 - EPSILON_RANGE)

print(f"D range: [{minimumD}, {maximumD}]")

D = 70 / 100

portfolio = VaultAllocation(
    df=pd.DataFrame(),
    mcaps=None,
    asset_weight_bounds={"*": (0, 1)}
)


class CustomVaultAllocationDelegate(GeneralOptimizationDelegate):

    def setup(self, optimization_object: VaultAllocation):
        optimization_object.yield_data = mu
        optimization_object.recovery_rate = R
        optimization_object.default_risk = lmb
        optimization_object.risk = omega
        optimization_object.diversification = D
        return super().setup(optimization_object)


portfolio.delegate = CustomVaultAllocationDelegate()

pf = portfolio.get_weights()

print(portfolio.get_metrics())

portfolio_plot = np.reshape(pf, [-1, 1])
portfolio_plot = pd.DataFrame(portfolio_plot, columns=np.array(["w"]))
portfolio_plot.index = np.arange(0, len(mu.index))

portfolio_plot["w"] = np.round(pf * 100, 2)
portfolio_plot["mu"] = np.round(mu * 100, 2)

ax = sns.barplot(
    x="w", y=portfolio_plot.index, data=portfolio_plot, orient="h", saturation=0.7
)

ax.set(xlabel="weight (%)", ylabel="vault")

mu (%): 0    4.0
1    4.0
2    4.0
dtype: float64
 
Recovery (%): 0    50.0
1    50.0
2    50.0
dtype: float64
 
lambda (%): 0    50.0
1    50.0
2    50.0
dtype: float64
 
D range: [33.336666666666666, 99.99]


ValueError: f(a) and f(b) must have different signs